In [ ]:
import os
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import uuid
import re
from dotenv import load_dotenv
import time

# 환경변수 로드
load_dotenv()

class PerfumeEmbedder:
    def __init__(self):
        # Pinecone 초기화
        self.pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
        self.index_name = "perfume-vec2"
        
        # 임베딩 모델 초기화 (다국어 지원 모델)
        self.model = SentenceTransformer('jhgan/ko-sroberta-multitask')
        
        # 인덱스 생성 또는 연결
        self._setup_index()
    
    def _setup_index(self):
        """Pinecone 인덱스 설정"""
        # 기존 인덱스 확인
        existing_indexes = [index.name for index in self.pc.list_indexes()]
        
        if self.index_name not in existing_indexes:
            # 인덱스 생성 (임베딩 차원: ko-sroberta-multitask는 768차원)
            self.pc.create_index(
                name=self.index_name,
                dimension=768,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                )
            )
            print(f"인덱스 '{self.index_name}' 생성 완료")
            time.sleep(10)  # 인덱스 생성 대기
        
        self.index = self.pc.Index(self.index_name)
        print(f"인덱스 '{self.index_name}' 연결 완료")
    
    def parse_notes_score(self, notes_score):
        """notes_score를 파싱해서 주요 향조 리스트 반환"""
        if pd.isna(notes_score):
            return []
        
        # "fruity(100.0) / sweet(68.4) / woody(66.0)" 형태를 파싱
        pattern = r'(\w+)\([\d.]+\)'
        matches = re.findall(pattern, notes_score)
        return matches
    
    def create_embedding_text(self, row):
        """각 향수 데이터를 임베딩용 텍스트로 변환"""
        text_parts = []
        
        # 기본 정보
        text_parts.append(f"향수명: {row['name']}")
        text_parts.append(f"영문명: {row['eng_name']}")
        text_parts.append(f"브랜드: {row['brand']}")
        
        # 향 정보
        if not pd.isna(row['main_accords']):
            text_parts.append(f"주요향조: {row['main_accords']}")
        
        if not pd.isna(row['top_notes']):
            text_parts.append(f"탑노트: {row['top_notes']}")
            
        if not pd.isna(row['middle_notes']):
            text_parts.append(f"미들노트: {row['middle_notes']}")
            
        if not pd.isna(row['base_notes']):
            text_parts.append(f"베이스노트: {row['base_notes']}")
        
        # 설명
        if not pd.isna(row['description']):
            text_parts.append(f"설명: {row['description']}")
        
        # 향 특성 (상위 5개만)
        notes_list = self.parse_notes_score(row['notes_score'])
        if notes_list:
            top_notes = notes_list[:5]  # 상위 5개 향조
            text_parts.append(f"향 특성: {', '.join(top_notes)}")
        
        # 시즌 정보
        if not pd.isna(row['season_score']):
            season_info = self.parse_season_info(row['season_score'])
            if season_info:
                text_parts.append(f"적합시즌: {season_info}")
        
        # 시간대 정보
        if not pd.isna(row['day_night_score']):
            time_info = self.parse_time_info(row['day_night_score'])
            if time_info:
                text_parts.append(f"적합시간: {time_info}")
        
        return " | ".join(text_parts)
    
    def parse_season_info(self, season_score):
        """시즌 점수를 파싱해서 가장 높은 시즌 반환"""
        if pd.isna(season_score):
            return ""
        
        # "winter(11.2) / spring(11.3) / summer(3.5) / fall(43.7)" 파싱
        seasons = {'winter': '겨울', 'spring': '봄', 'summer': '여름', 'fall': '가을'}
        pattern = r'(\w+)\(([\d.]+)\)'
        matches = re.findall(pattern, season_score)
        
        if matches:
            best_season = max(matches, key=lambda x: float(x[1]))
            return seasons.get(best_season[0], best_season[0])
        return ""
    
    def parse_time_info(self, day_night_score):
        """시간대 점수를 파싱"""
        if pd.isna(day_night_score):
            return ""
        
        # "day(97.5) / night(99.3)" 파싱
        pattern = r'day\(([\d.]+)\) / night\(([\d.]+)\)'
        match = re.search(pattern, day_night_score)
        
        if match:
            day_score = float(match.group(1))
            night_score = float(match.group(2))
            
            if day_score > night_score:
                return "주간용"
            elif night_score > day_score:
                return "야간용"
            else:
                return "올데이"
        return ""
    
    def prepare_metadata(self, row):
        """메타데이터 준비 (필터링용)"""
        metadata = {
            'brand': row['brand'],
            'name': row['name'],
            'eng_name': row['eng_name'],
            'size_ml': int(row['size_ml']) if not pd.isna(row['size_ml']) else 0,
            'price_krw': int(row['price_krw']) if not pd.isna(row['price_krw']) else 0,
            'concentration': row['concentration'] if not pd.isna(row['concentration']) else '',
            'gender': row['gender'] if not pd.isna(row['gender']) else '',
            'detail_url': row['detail_url'] if not pd.isna(row['detail_url']) else ''
        }
        
        # 주요 향조 리스트 추가
        notes_list = self.parse_notes_score(row['notes_score'])
        if notes_list:
            metadata['main_notes'] = notes_list[:5]  # 상위 5개
        
        # 시즌 정보 추가
        season_info = self.parse_season_info(row['season_score'])
        if season_info:
            metadata['best_season'] = season_info
            
        # 시간대 정보 추가
        time_info = self.parse_time_info(row['day_night_score'])
        if time_info:
            metadata['best_time'] = time_info
        
        return metadata
    
    def upload_perfumes(self, csv_file_path, batch_size=100):
        """CSV 파일에서 향수 데이터를 읽어 Pinecone에 업로드"""
        
        # CSV 파일 읽기
        df = pd.read_csv(csv_file_path)
        print(f"총 {len(df)}개의 향수 데이터 로드")
        
        vectors_to_upsert = []
        
        for idx, row in df.iterrows():
            try:
                # 임베딩 텍스트 생성
                embedding_text = self.create_embedding_text(row)
                
                # 임베딩 생성
                embedding = self.model.encode(embedding_text).tolist()
                
                # 메타데이터 준비
                metadata = self.prepare_metadata(row)
                
                # 벡터 ID 생성 (브랜드_향수명_사이즈)
                vector_id = f"{row['brand']}_{row['name']}_{row['size_ml']}ml"
                vector_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, vector_id))
                
                vector_data = {
                    'id': vector_id,
                    'values': embedding,
                    'metadata': metadata
                }
                
                vectors_to_upsert.append(vector_data)
                
                # 배치 사이즈마다 업로드
                if len(vectors_to_upsert) >= batch_size:
                    self.index.upsert(vectors=vectors_to_upsert)
                    print(f"{idx + 1}개 데이터 업로드 완료...")
                    vectors_to_upsert = []
                    time.sleep(1)  # API 제한 방지
                    
            except Exception as e:
                print(f"오류 발생 (행 {idx}): {e}")
                continue
        
        # 남은 벡터들 업로드
        if vectors_to_upsert:
            self.index.upsert(vectors=vectors_to_upsert)
            print(f"최종 {len(vectors_to_upsert)}개 데이터 업로드 완료")
        
        # 인덱스 통계 확인
        stats = self.index.describe_index_stats()
        print(f"\n업로드 완료! 총 벡터 수: {stats['total_vector_count']}")

# 사용 예시
if __name__ == "__main__":
    # PerfumeEmbedder 인스턴스 생성
    embedder = PerfumeEmbedder()
    
    # CSV 파일 경로
    csv_file_path = "perfume_final_gender_completed_full.csv"
    
    # 데이터 업로드
    embedder.upload_perfumes(csv_file_path)
    
    print("향수 데이터 업로드가 완료되었습니다!")

인덱스 'perfume-search' 연결 완료
총 1395개의 향수 데이터 로드
100개 데이터 업로드 완료...
200개 데이터 업로드 완료...
300개 데이터 업로드 완료...
400개 데이터 업로드 완료...
500개 데이터 업로드 완료...
600개 데이터 업로드 완료...
700개 데이터 업로드 완료...
800개 데이터 업로드 완료...
900개 데이터 업로드 완료...
1000개 데이터 업로드 완료...
1100개 데이터 업로드 완료...
1200개 데이터 업로드 완료...
1300개 데이터 업로드 완료...
최종 95개 데이터 업로드 완료

업로드 완료! 총 벡터 수: 1238
향수 데이터 업로드가 완료되었습니다!
